In [2]:
!pip install biopython torch

     |████████████████████████████████| 2.3MB 2.7MB/s 


In [0]:
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
from Bio import SeqIO
import re

def load_fasta(fname):
  parsed = []
  for seq in SeqIO.parse(fname, "fasta"):
    record = { 'seq': str(seq.seq) }
    if 'trueSeq' in seq.id:
      meta = re.search('::\w+::(?P<rbps>(?:\w+(?:///)?)+)::(?P<id>\w+)', seq.id).groupdict()
      rbps = {key: 1 for key in meta['rbps'].split('///')}
      record['id'] = meta['id']
      record.update(rbps)
    parsed.append(record)
  return pd.DataFrame(parsed).fillna(0)

train = load_fasta('train.txt')
valid = load_fasta('valid.txt')
test = load_fasta('test.txt')

In [56]:
train.head()

,seq,id,ATXN2,FUS,CPSF6,HNRNPC,NUDT21,ELAVL1,LIN28B,FIP1L1,MOV10,WDR33,CPSF7,YTHDF2,CSTF2T,ZC3H7B,CAPRIN1,CPSF1,CPSF4,CPSF3,TARDBP,CSTF2
0,agcatgttagagttctgatggcagttgttgtttatgaataagCTTC...,ENST00000545308,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,aaacatattctattcatttattttataactgttcccttgacctctt...,ENST00000606457,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ccccacccacccctttggagaggtgtccctcgtaacattctggagc...,ENST00000506172,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,cttgcagtctgagcacgcgtaaggcttctccccggtgtgagcccgc...,ENST00000492040,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,taatccacagtttcccaacctcagccctactgacactggggctgga...,ENST00000572913,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
train.tail()

,seq,id,ATXN2,FUS,CPSF6,HNRNPC,NUDT21,ELAVL1,LIN28B,FIP1L1,MOV10,WDR33,CPSF7,YTHDF2,CSTF2T,ZC3H7B,CAPRIN1,CPSF1,CPSF4,CPSF3,TARDBP,CSTF2
194978,TTTTGTTTCCTGTTAATCTTTGTATCTAGGAAAATCTAATCTGCCT...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194979,GCCTTTTATGGGCCACTTGCATTAGTGTCCTGATAGAGGCTTAATC...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194980,GTCTGTCTTCTGTATATACCCTGTAGATCCGAATTTGTGTAAGGAA...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194981,GTTCAGGGTACAGCCAGGTGGGGGAGACAGATCACTATAAAAACCT...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194982,ATGTAGCCAAAATCAGCCAGGAGCGGTGGCTCACCCCTGTAATCCC...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
